In [8]:

!pip install classiq==0.70.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from classiq import (
    create_model,
    Circuit,
    H,
    X,
    Z,
    Measure
)
import random
import numpy as np

class BB84Protocol:
    def __init__(self, num_qubits=10):
        """
        Initialize BB84 Quantum Key Distribution Protocol
        
        Args:
            num_qubits (int): Number of qubits to transmit
        """
        self.num_qubits = num_qubits
        self.alice_bases = []
        self.alice_bits = []
        self.bob_bases = []
        self.bob_bits = []
        
    def create_bb84_circuit(self):
        """
        Create quantum circuit for BB84 protocol
        
        Returns:
            Quantum circuit
        """
        @create_model
        def bb84_circuit(target_qubits):
            # Generate random bits and bases for Alice
            for i in range(self.num_qubits):
                # Randomly choose bit (0 or 1)
                bit = random.randint(0, 1)
                # Randomly choose measurement basis
                basis = random.choice(['rectilinear', 'diagonal'])
                
                self.alice_bits.append(bit)
                self.alice_bases.append(basis)
                
                # Prepare quantum state based on bit and basis
                if basis == 'rectilinear':
                    # Computational basis (Z basis)
                    if bit == 1:
                        X(target_qubits[i])
                else:
                    # Diagonal basis (X basis)
                    H(target_qubits[i])
                    if bit == 1:
                        Z(target_qubits[i])
            
            # Bob's measurement bases
            bob_bases = [random.choice(['rectilinear', 'diagonal']) for _ in range(self.num_qubits)]
            self.bob_bases = bob_bases
            
            # Bob's measurement
            for i in range(self.num_qubits):
                # Change basis if needed
                if bob_bases[i] == 'diagonal':
                    H(target_qubits[i])
                
                # Measure the qubit
                self.bob_bits.append(Measure(target_qubits[i]))
            
            return target_qubits
        
        return bb84_circuit
    
    def sift_keys(self):
        """
        Sift the keys by comparing bases
        
        Returns:
            Tuple of matching keys for Alice and Bob
        """
        alice_sifted_key = []
        bob_sifted_key = []
        
        for i in range(self.num_qubits):
            if self.alice_bases[i] == self.bob_bases[i]:
                alice_sifted_key.append(self.alice_bits[i])
                bob_sifted_key.append(self.bob_bits[i])
        
        return alice_sifted_key, bob_sifted_key
    
    def run_protocol(self):
        """
        Execute the full BB84 protocol
        
        Returns:
            Tuple of results: (Alice's key, Bob's key, Matching percentage)
        """
        # Prepare the quantum circuit
        bb84_model = self.create_bb84_circuit()
        
        # Sift the keys
        alice_key, bob_key = self.sift_keys()
        
        # Calculate matching percentage
        matching_percentage = sum(a == b for a, b in zip(alice_key, bob_key)) / len(alice_key) * 100 if alice_key else 0
        
        return alice_key, bob_key, matching_percentage

def main():
    # Run BB84 protocol
    protocol = BB84Protocol(num_qubits=20)
    alice_key, bob_key, matching_percentage = protocol.run_protocol()
    
    print("Alice's Bases:", protocol.alice_bases)
    print("Bob's Bases: ", protocol.bob_bases)
    print("\nAlice's Key:", alice_key)
    print("Bob's Key:  ", bob_key)
    print(f"\nKey Matching Percentage: {matching_percentage:.2f}%")

if __name__ == "__main__":
    main()

ImportError: cannot import name 'Circuit' from 'classiq' (c:\Users\Aravind\college\quantum\venv\Lib\site-packages\classiq\__init__.py)

In [12]:
import classiq
print(classiq.__version__)
print(dir(classiq))

0.70.0
['Analyzer', 'Array', 'CArray', 'CBool', 'CCX', 'CH', 'CInt', 'CPHASE', 'CRX', 'CRY', 'CRZ', 'CReal', 'CX', 'CY', 'CZ', 'ChemistryAtom', 'CombinatorialOptimizationSolution', 'Constraints', 'ControlState', 'CustomHardwareSettings', 'Element', 'ExecutionParams', 'FermionMapping', 'FinanceFunction', 'FinanceFunctionType', 'FockHamiltonianProblem', 'GaussianModel', 'H', 'I', 'IDENTITY', 'Input', 'LadderOp', 'LadderOperator', 'LadderTerm', 'LogNormalModel', 'Molecule', 'MoleculeProblem', 'OptimizationParameter', 'Optimizer', 'Output', 'PHASE', 'Pauli', 'PauliTerm', 'Position', 'Preferences', 'QArray', 'QBit', 'QCallable', 'QCallableList', 'QConstant', 'QNum', 'QSVMFeatureMapEntanglement', 'QSVMFeatureMapPauli', 'QStruct', 'QsvmResult', 'QuantumProgram', 'R', 'RESET', 'RX', 'RXX', 'RY', 'RYY', 'RZ', 'RZZ', 'RegisterArithmeticInfo', 'RegisterUserInput', 'S', 'SDG', 'SIGNED', 'SWAP', 'T', 'TDG', 'TranspilerBasisGates', 'U', 'UNSIGNED', 'X', 'Y', 'Z', '_single_pauli', 'add', 'allocate', 

In [13]:
from classiq import (
    create_model,
    QArray,
    QBit,
    H,
    X,
    Z,
    execute,
    synthesize
)
import random
import numpy as np

class BB84Protocol:
    def __init__(self, num_qubits=10):
        """
        Initialize BB84 Quantum Key Distribution Protocol
        
        Args:
            num_qubits (int): Number of qubits to transmit
        """
        self.num_qubits = num_qubits
        self.alice_bases = []
        self.alice_bits = []
        self.bob_bases = []
        self.bob_bits = []
        
    def create_bb84_circuit(self):
        """
        Create quantum circuit for BB84 protocol
        
        Returns:
            Quantum circuit model
        """
        @create_model
        def bb84_circuit(qubits: QArray[QBit]):
            # Reset tracking lists
            self.alice_bases.clear()
            self.alice_bits.clear()
            self.bob_bases.clear()
            self.bob_bits.clear()
            
            # Alice prepares qubits
            for i in range(self.num_qubits):
                # Generate random bit and basis
                bit = random.randint(0, 1)
                basis = random.choice(['rectilinear', 'diagonal'])
                
                self.alice_bits.append(bit)
                self.alice_bases.append(basis)
                
                # Prepare quantum state based on bit and basis
                if basis == 'rectilinear':
                    # Computational basis (Z basis)
                    if bit == 1:
                        X(qubits[i])
                else:
                    # Diagonal basis (X basis)
                    H(qubits[i])
                    if bit == 1:
                        Z(qubits[i])
            
            # Bob's measurement bases
            bob_bases = [random.choice(['rectilinear', 'diagonal']) for _ in range(self.num_qubits)]
            self.bob_bases = bob_bases
            
            # Simulate measurement (in a real quantum system, this would be a true quantum measurement)
            for i in range(self.num_qubits):
                # Change basis if needed
                if bob_bases[i] == 'diagonal':
                    H(qubits[i])
                
                # In a simulation, we'll use the original bit with some noise
                bit = self.alice_bits[i] if random.random() > 0.1 else 1 - self.alice_bits[i]
                self.bob_bits.append(bit)
            
            return qubits
        
        return bb84_circuit
    
    def sift_keys(self):
        """
        Sift the keys by comparing bases
        
        Returns:
            Tuple of matching keys for Alice and Bob
        """
        alice_sifted_key = []
        bob_sifted_key = []
        
        for i in range(self.num_qubits):
            if self.alice_bases[i] == self.bob_bases[i]:
                alice_sifted_key.append(self.alice_bits[i])
                bob_sifted_key.append(self.bob_bits[i])
        
        return alice_sifted_key, bob_sifted_key
    
    def run_protocol(self):
        """
        Execute the full BB84 protocol
        
        Returns:
            Tuple of results: (Alice's key, Bob's key, Matching percentage)
        """
        # Create the quantum circuit model
        bb84_model = self.create_bb84_circuit()
        
        # Synthesize the circuit
        circuit = synthesize(bb84_model)
        
        # Execute the circuit (this is a simulation)
        execute(circuit)
        
        # Sift keys (matching bases)
        alice_key, bob_key = self.sift_keys()
        
        # Calculate matching percentage
        matching_percentage = (
            sum(a == b for a, b in zip(alice_key, bob_key)) / 
            len(alice_key) * 100 if alice_key else 0
        )
        
        return alice_key, bob_key, matching_percentage

def main():
    # Run BB84 protocol
    protocol = BB84Protocol(num_qubits=20)
    alice_key, bob_key, matching_percentage = protocol.run_protocol()
    
    print("Alice's Bases:", protocol.alice_bases)
    print("Bob's Bases: ", protocol.bob_bases)
    print("\nAlice's Key:", alice_key)
    print("Bob's Key:  ", bob_key)
    print(f"\nKey Matching Percentage: {matching_percentage:.2f}%")

if __name__ == "__main__":
    main()

AttributeError: 'function' object has no attribute 'func_decl'

In [18]:
from classiq import ModelBuilder
import numpy as np

# Number of qubits (key length)
n = 10  # Change as needed

# Step 1: Alice generates random bits and bases
alice_bits = np.random.randint(2, size=n)  # 0s and 1s
alice_bases = np.random.randint(2, size=n)  # 0: Standard basis | 1: Hadamard basis

# Step 2: Define the quantum circuit using Classiq
builder = ModelBuilder()
qubits = builder.add_qubits(n)

# Alice prepares qubits based on bits and bases
for i in range(n):
    if alice_bits[i] == 1:
        builder.apply_gate("X", qubits[i])  # Encode bit as |1>
    if alice_bases[i] == 1:
        builder.apply_gate("H", qubits[i])  # Apply H if in Hadamard basis

# Step 3: Bob selects random bases for measurement
bob_bases = np.random.randint(2, size=n)
for i in range(n):
    if bob_bases[i] == 1:
        builder.apply_gate("H", qubits[i])  # Convert to standard basis if Hadamard
    builder.measure(qubits[i])

# Compile the quantum model
builder.compile()

# Step 4: Basis reconciliation (Key Sifting)
matching_bases = alice_bases == bob_bases
sifted_key = alice_bits[matching_bases]

# Step 5: Eavesdropping detection (Comparing a subset)
sample_size = max(1, len(sifted_key) // 4)  # Take 25% as sample
sample_indices = np.random.choice(len(sifted_key), sample_size, replace=False)
sample_bits = sifted_key[sample_indices]

# Alice and Bob verify sample bits to detect eavesdropping
if np.any(sample_bits != sifted_key[sample_indices]):
    print("Eavesdropper detected! Key discarded.")
else:
    print("Key is secure. Final key:", sifted_key)


ImportError: cannot import name 'ModelBuilder' from 'classiq' (c:\Users\Aravind\college\quantum\venv\Lib\site-packages\classiq\__init__.py)